In [20]:
import os
import pandas as pd
from glob import glob
from tqdm import tqdm
from scipy import io
from scipy.sparse import csr_matrix
import numpy as np
import shutil

DATA_NAME = "PBMC10k"
WINDOW = 2000
COACC_THRESH = 0.1

base = f"../../Datasets/{DATA_NAME}"
cicero_dir = f"{base}/output/cicero/filtered"
core_file = f"{base}/output/celltypist/core_genes_regions_{WINDOW}bp_intersected_peaks.txt"
out_file = f"{base}/output/celltypist_cicero/selected_peaks_{WINDOW}bp_coacc{COACC_THRESH}.txt"

In [16]:
# --- Читаем ядро ----
with open(core_file) as f:
    core_peak_ids = {line.strip().strip('"') for line in f if line.strip()}

print("Ядерных пиков:", len(core_peak_ids))

Ядерных пиков: 1922


In [17]:
# --- Находим файлы со связями по хромосомам ---
files = sorted(glob(os.path.join(cicero_dir, "peaks_chr*.csv")))
print("Найдено файлов:", len(files))

Найдено файлов: 23


In [18]:
selected_peak_ids = set(core_peak_ids)

total_connections = 0
core_connections = 0

for file in tqdm(files):
    df = pd.read_csv(file)
    total_connections += df.shape[0]

    # Уберём кавычки у Peak1/Peak2 — вдруг они там встречаются
    df["Peak1"] = df["Peak1"].astype(str).str.replace('"', '')
    df["Peak2"] = df["Peak2"].astype(str).str.replace('"', '')

    # фильтруем по порогу coaccess
    df_thr = df[df["coaccess"] >= COACC_THRESH]

    # оставляем только связи, где хотя бы один пик из ядра
    mask = df_thr["Peak1"].isin(core_peak_ids) | df_thr["Peak2"].isin(core_peak_ids)
    df_core = df_thr[mask]

    core_connections += df_core.shape[0]

    # добавляем оба пика
    selected_peak_ids.update(df_core["Peak1"].tolist())
    selected_peak_ids.update(df_core["Peak2"].tolist())

print(f"Всего связей во всех файлах: {total_connections}")
print(f"Связей ядро–coaccessible: {core_connections}")
print(f"Итоговое число пиков (ядро + coaccessible): {len(selected_peak_ids)}")

100%|████████████████████| 23/23 [00:00<00:00, 23.59it/s]

Всего связей во всех файлах: 845962
Связей ядро–coaccessible: 39334
Итоговое число пиков (ядро + coaccessible): 15208


In [19]:
# --- Сохраняем ---
with open(out_file, "w") as f:
    for pid in sorted(selected_peak_ids):
        f.write(pid + "\n")

print("Сохранено в:", out_file)

Сохранено в: ../../Datasets/PBMC10k/output/celltypist_cicero/selected_peaks_2000bp_coacc0.1.txt


In [22]:
cicero_dir = f"{base}/output/cicero"
out_dir = f"{base}/output/celltypist_cicero"

peaks_ids_path = os.path.join(cicero_dir, "peaks.txt")
with open(peaks_ids_path) as f:
    all_peak_ids = [line.strip() for line in f if line.strip()]

print("Всего пиков в исходной матрице:", len(all_peak_ids))

# индексы строк (пиков), которые мы хотим оставить
idx = [i for i, pid in enumerate(all_peak_ids) if pid in selected_peak_ids]
print("Пиков, совпавших с selected_ids:", len(idx))

if not idx:
    raise RuntimeError("Ни один peak_id из selected_ids не найден в peaks.txt — проверь формат ID.")

Всего пиков в исходной матрице: 49481
Пиков, совпавших с selected_ids: 14670


In [23]:
idx = np.array(idx, dtype=int)

# --- режем матрицу ---
mat_path = os.path.join(cicero_dir, "matrix.mtx")
mat = io.mmread(mat_path).tocsr()
print("Исходная матрица:", mat.shape)    # (n_peaks, n_cells)

Исходная матрица: (49481, 10032)


In [24]:
mat_core = mat[idx, :]
print("Урезанная матрица:", mat_core.shape)

Урезанная матрица: (14670, 10032)


In [27]:
# --- сохраняем новый набор для scOpen ---

# 1) peaks.txt — только выбранные пики, в том порядке, в каком они идут в матрице
core_peak_ids_ordered = [all_peak_ids[i] for i in idx]
with open(os.path.join(out_dir, "peaks.txt"), "w") as f:
    for pid in core_peak_ids_ordered:
        f.write(pid + "\n")

with open(os.path.join(out_dir, "peaks.bed"), "w") as f:
    for pid in core_peak_ids_ordered:
        f.write(pid.replace('_', '\t') + "\n")

# 2) barcodes.tsv копируем без изменений (клетки не меняем)
shutil.copy(
    os.path.join(cicero_dir, "barcodes.tsv"),
    os.path.join(out_dir, "barcodes.tsv")
)

# 3) matrix.mtx — урезанная по пикам матрица
io.mmwrite(os.path.join(out_dir, "matrix.mtx"), mat_core)

print("Новый инпут для scOpen сохранён в:", out_dir)

Новый инпут для scOpen сохранён в: ../../Datasets/PBMC10k/output/celltypist_cicero


In [26]:
pid

'chrX_154056596_154057416'